In [1]:
import copy
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F

from datetime import datetime
from lamah_dataset import LamaHDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import to_networkx
from torchinfo import summary
from tqdm import tqdm

In [5]:
def ensure_reproducibility(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def train_step(model, train_loader, optimizer, device):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc="Training"):
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = F.mse_loss(out, batch.y)
        train_loss += loss.item() * batch.num_graphs / len(train_dataset)
        loss.backward()
        optimizer.step()
    return train_loss


def eval_step(model, eval_loader, device):
    model.eval()
    eval_loss = 0
    with torch.no_grad():
        for batch in tqdm(eval_loader, desc="Evaluating"):
            batch = batch.to(device)
            out = model(batch)
            loss = F.mse_loss(out, batch.y)
            eval_loss += loss.item() * batch.num_graphs / len(eval_dataset)
    return eval_loss


def train(model, train_dataset, eval_dataset, hparams):
    train_loader = DataLoader(train_dataset, batch_size=hparams["training"]["batch_size"], shuffle=True)
    eval_loader = DataLoader(eval_dataset, batch_size=hparams["training"]["batch_size"], shuffle=False)
    optimizer = torch.optim.Adam(model.parameters(),
                             lr=hparams["training"]["learning_rate"],
                             weight_decay=hparams["training"]["weight_decay"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)
    model = model.to(device)

    history = {"train_loss": [], "eval_loss": [], "model_params": [], "optim_params": []}

    for epoch in range(hparams["training"]["num_epochs"]):
        print(f"Epoch {epoch + 1}/{hparams['training']['num_epochs']}")

        train_loss = train_step(model, train_loader, optimizer, device)
        history["train_loss"].append(train_loss)

        eval_loss = eval_step(model, eval_loader, device)
        history["eval_loss"].append(eval_loss)

        history["model_params"].append(copy.deepcopy(model.state_dict()))
        history["optim_params"].append(copy.deepcopy(optimizer.state_dict()))

        print(f"Loss (train/eval): {train_loss:.4f}/{eval_loss:.4f}")

    torch.save({
        "history": history,
        "hparams": hparams
    }, datetime.now().strftime("%Y-%m-%d_%H-%M-%S_" + model.__class__.__name__))

    return history

In [6]:
def get_edge_weights(weight_type):
    if weight_type == "binary":
        return torch.ones(data.num_edges)
    if weight_type == "dist_hdn":
        return data.edge_attr[:, 0]
    elif weight_type == "elev_diff":
        return data.edge_attr[:, 1]
    elif weight_type == "strm_slope":
        return data.edge_attr[:, 2]
    elif weight_type == "disabled":
        return torch.zeros(data.num_edges)
    elif weight_type == "learned":
        return nn.Parameter(torch.ones(data.num_edges))
    else:
        raise ValueError("Invalid weight type given!")


class MLP(nn.Module):
    def __init__(self, window_size_hrs=24, num_hidden=2, hidden_size=128, residual=False):
        super().__init__()
        self.residual = residual
        self.layers = nn.ModuleList([nn.Linear(window_size_hrs, hidden_size) if i == 0 else
                                     nn.Linear(hidden_size, 1) if i == num_hidden else
                                     nn.Linear(hidden_size, hidden_size)
                                     for i in range(0, num_hidden + 1)])

    def forward(self, data):
        x = data.x

        x = F.relu(self.layers[0](x))
        for layer in self.layers[1:-1]:
            x = F.relu(layer(x)) + (x if self.residual else 0)
        x = self.layers[-1](x)

        return x


class GCN(nn.Module):
    def __init__(self, window_size_hrs=24, num_convs=2, hidden_size=128, residual=False, weight_type="binary"):
        super().__init__()
        self.dense_in = nn.Linear(window_size_hrs, hidden_size)
        self.convs = nn.ModuleList([GCNConv(hidden_size, hidden_size)
                                    for _ in range(num_convs)])
        self.dense_out = nn.Linear(hidden_size, 1)
        self.residual = residual
        self.edge_weights = get_edge_weights(weight_type)

    def forward(self, data):
        x, edge_idx = data.x, data.edge_index
        edge_w = self.edge_weights.repeat(data.num_graphs)

        x = F.relu(self.dense_in(x))
        for conv in self.convs:
            x = F.relu(conv(x, edge_idx, edge_w)) \
                + (x if self.residual else 0)
        x = self.dense_out(x)

        return x


class GAT(torch.nn.Module):
    def __init__(self, window_size_hrs=24, num_hidden=2, hidden_size=128):
        super().__init__()
        self.layers = nn.ModuleList([GATConv(window_size_hrs, hidden_size) if i == 0 else
                                     GATConv(hidden_size, 1) if i == num_hidden else
                                     GATConv(hidden_size, hidden_size)
                                     for i in range(0, num_hidden + 1)])

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        for layer in self.layers[:-1]:
            x = layer(x, edge_index)
            x = F.elu(x)
        x = self.layers[-1](x, edge_index)

        return x

In [ ]:
ensure_reproducibility(123456789)

In [3]:
HPARAMS = {
    "data": {
        "window_size": 24,
        "stride_length": 1,
        "lead_time": 1,
        "bidirectional": False,
        "normalized": True
    },
    "model": {
        "propagation_dist": 19,
        "hidden_size": 128,
        "residual": True,
        "weight_type": "binary"
    },
    "training": {
        "num_epochs": 50,
        "batch_size": 32,
        "learning_rate": 0.005,
        "weight_decay": 0  # 5e-4
    }
}

Loading dataset into memory...


100%|██████████| 375/375 [00:20<00:00, 18.10it/s]


In [ ]:
dataset = LamaHDataset("LamaH-CE",
                       years=range(2000, 2018),
                       window_size_hrs=HPARAMS["data"]["window_size"],
                       stride_length_hrs=HPARAMS["data"]["stride_length"],
                       lead_time_hrs=HPARAMS["data"]["lead_time"],
                       bidirectional=HPARAMS["data"]["bidirectional"],
                       normalize=HPARAMS["data"]["normalized"])
train_dataset = dataset.subsample_years(y for y in range(2000, 2018) if y % 8 > 0)
eval_dataset = dataset.subsample_years(y for y in range(2000, 2018) if y % 8 == 0)

In [5]:
data = eval_dataset[4200]
print("# vertices:", data.num_nodes)
print("# edges:", data.num_edges)
print("# vertex features:", data.num_node_features)
print("# edge features:", data.num_edge_features)
print("# faces:", data.num_faces)
print("directed graph?:", data.is_directed())
print("isolated vertices?", data.has_isolated_nodes())
print("self-loops?", data.has_self_loops())
print("coalesced?", data.is_coalesced())
print("valid?", data.validate())
plt.figure(figsize=(12, 9))
graph = to_networkx(data, to_undirected=False)
nx.draw_networkx(graph, pos=nx.drawing.kamada_kawai_layout(graph), with_labels=True,
                 node_color=[tuple(torch.rand(3)) for _ in range(375)])

NameError: name 'eval_dataset' is not defined

In [ ]:
# TODO try F.elu at some point
#model = MLP(window_size_hrs=WINDOW_SIZE, num_hidden=19, hidden_size=128, residual=True).to(device)
model = GCN(window_size_hrs=HPARAMS["data"]["window_size"],
            num_convs=HPARAMS["model"]["propagation_dist"],
            hidden_size=HPARAMS["model"]["hidden_size"],
            residual=HPARAMS["model"]["residual"],
            weight_type=HPARAMS["model"]["weight_type"])
print(summary(model, depth=2))

history = train(model, train_dataset, eval_dataset, HPARAMS)

In [ ]:
plt.plot(history["train_loss"], label="train")
plt.plot(history["eval_loss"], label="eval")
plt.xlabel("epoch")
plt.ylabel("MSE")
plt.legend()
i = history["eval_loss"].index(min(history["eval_loss"]))
print(f"{history['train_loss'][i]:.4f}/{history['eval_loss'][i]:.4f}")